In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import string

In [2]:
path_train = "../input/covid-19-nlp-text-classification/Corona_NLP_train.csv"
path_test = "../input/covid-19-nlp-text-classification/Corona_NLP_test.csv"
df_train = pd.read_csv(path_train, encoding = 'latin1')
df_test = pd.read_csv(path_test, encoding = 'latin1')

In [3]:
df_train.drop(['UserName', 'ScreenName', 'Location', 'TweetAt'], axis = 1, inplace = True)
df_test.drop(['UserName', 'ScreenName', 'Location', 'TweetAt'], axis = 1, inplace = True)
df_train = df_train[(df_train.Sentiment == 'Extremely Positive') | (df_train.Sentiment == 'Positive') | (df_train.Sentiment == 'Extremely Negative') | (df_train.Sentiment == 'Negative')]
df_test = df_test[(df_test.Sentiment == 'Extremely Positive') | (df_test.Sentiment == 'Positive') | (df_test.Sentiment == 'Extremely Negative') | (df_test.Sentiment == 'Negative')]

In [4]:
max_seq_len = 140
max_features = 30000
stop_words = stopwords.words("english")

def makediglabel(text):
    if text in ['Extremely Positive', 'Positive']:
        return 0.0
    elif text in ['Extremely Negative', 'Negative']:
        return 1.0
    else:
        return -1.0
    
def clean_text(text):
    text = text.lower()
    text = re.sub("@\S+", " ", text)  
    text = re.sub("https*\S+", " ", text)
    text = re.sub("www\S+", " ", text)
    text = re.sub("#\S+", " ", text)
    text = re.sub("\d", " ", text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\n', ' ', text)        
    text = re.sub('\s{2,}',' ', text)
    text = ' '.join([word for word in text.split(' ') if word not in stop_words])
    return text

def convert(text):
    tok = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    lem = [lemmatizer.lemmatize(t) for t in tok]
    res = [dictionary.index(i) for i in lem]
    return res

In [5]:
df_train.OriginalTweet = df_train.OriginalTweet.apply(clean_text)
df_test.OriginalTweet = df_test.OriginalTweet.apply(clean_text)
df_train.Sentiment = df_train.Sentiment.apply(makediglabel)
df_test.Sentiment = df_test.Sentiment.apply(makediglabel)

In [6]:
X_train = df_train.OriginalTweet
Y_train = df_train.Sentiment
X_test = df_test.OriginalTweet
Y_test = df_test.Sentiment

In [7]:
whole_text = ""
for i in X_train:
    whole_text += i + ' '
for i in X_test:
    whole_text += i + ' '
tokens = word_tokenize(whole_text)
lemmatizer = WordNetLemmatizer()
lemtok = [lemmatizer.lemmatize(t) for t in tokens]
dictionary = list(set(lemtok))

In [8]:
X_train = X_train.apply(convert)
X_test = X_test.apply(convert)
x_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen = max_seq_len)
x_test = keras.preprocessing.sequence.pad_sequences(X_test, maxlen = max_seq_len)
y_train = np.asarray(Y_train).astype("float32")
y_test = np.asarray(Y_test).astype("float32")

In [9]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [10]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [11]:
embed_dim = 32  # Embedding size for each token
num_heads = 4  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(max_seq_len,))
embedding_layer = TokenAndPositionEmbedding(max_seq_len, max_features, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(32, activation="relu")(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

2022-02-02 19:55:15.368668: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-02 19:55:15.492311: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-02 19:55:15.493409: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-02 19:55:15.501472: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [12]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    x_train, y_train, batch_size=32, epochs=3, validation_data=(x_test, y_test)
)

2022-02-02 19:55:18.224699: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/3
1046/1046 [==============================] - 10s 8ms/step - loss: 0.6000 - accuracy: 0.6312 - val_loss: 0.4040 - val_accuracy: 0.8135
Epoch 2/3
1046/1046 [==============================] - 8s 7ms/step - loss: 0.2755 - accuracy: 0.8917 - val_loss: 0.3208 - val_accuracy: 0.8606
Epoch 3/3
1046/1046 [==============================] - 8s 7ms/step - loss: 0.1743 - accuracy: 0.9362 - val_loss: 0.3266 - val_accuracy: 0.8685


In [13]:
score = model.evaluate(x_test, y_test, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])


Test score: 0.3265705704689026
Test accuracy: 0.8685120940208435
